### TODO

- Parameterize EVT (must first instantiate EVT pckg)
- Parameterize Reach 1 inflow
- Parameterize Lumprem models
- Covariance matrices

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
from flopy.utils.gridintersect import GridIntersect
import pyemu

from shapely.geometry import Polygon, Point
import shapefile
from shapely.prepared import prep

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.rcParams['font.size']=12

In [2]:
bins_pth = os.path.join('..', 'bins', 'win') if 'nt' in os.name else os.path.join('..', 'bins', 'linux') # Binaries
shapefile_pth = os.path.join('..', 'data', 'raw_data', 'shapefiles')
observations_pth = os.path.join('..', 'data', 'observations') # Measured data (field obs)

In [3]:
org_model_ws = os.path.join('..', 'temp_flopy_lumprem')
os.listdir(org_model_ws)

['ghbpp.dat',
 'ghbpp.tpl',
 'ghb_new.ts',
 'hagfors_1.cbb',
 'hagfors_1.disv',
 'hagfors_1.disv.grb',
 'hagfors_1.disv_botm.txt',
 'hagfors_1.disv_cell2d.txt',
 'hagfors_1.disv_top.txt',
 'hagfors_1.disv_vertices.txt',
 'hagfors_1.ghb',
 'hagfors_1.hds',
 'hagfors_1.ic',
 'hagfors_1.ic_strt.txt',
 'hagfors_1.ims',
 'hagfors_1.lst',
 'hagfors_1.nam',
 'hagfors_1.npf',
 'hagfors_1.npf_icelltype.txt',
 'hagfors_1.npf_k.txt',
 'hagfors_1.npf_k33.txt',
 'hagfors_1.obs',
 'hagfors_1.oc',
 'hagfors_1.rch',
 'hagfors_1.sfr',
 'hagfors_1.sfr.bud',
 'hagfors_1.sfr.obs',
 'hagfors_1.sto',
 'hagfors_1.sto_iconvert.txt',
 'hagfors_1.sto_ss.txt',
 'hagfors_1.sto_sy.txt',
 'hagfors_1.tdis',
 'head.obs.csv',
 'lr_blue.tpl',
 'lr_green.tpl',
 'lr_lu1.tpl',
 'lr_red.tpl',
 'lr_yellow.tpl',
 'mfsim.lst',
 'mfsim.nam',
 'rch_new.ts',
 'sfr_gage.obs.csv',
 'sfr_pred_leak.obs.csv',
 'sfr_reach1_inflow.ts',
 'sfr_reach_evaporation.ts',
 'sfr_reach_rainfall.ts',
 'sfr_rtp_initial.csv',
 'sfr_segfile.dat',
 '

In [4]:
tmp_model_ws = os.path.join('..', 'temp_pst_from')
if os.path.exists(tmp_model_ws):
    shutil.rmtree(tmp_model_ws)
shutil.copytree(org_model_ws,tmp_model_ws)
os.listdir(tmp_model_ws)

['ghbpp.dat',
 'ghbpp.tpl',
 'ghb_new.ts',
 'hagfors_1.cbb',
 'hagfors_1.disv',
 'hagfors_1.disv.grb',
 'hagfors_1.disv_botm.txt',
 'hagfors_1.disv_cell2d.txt',
 'hagfors_1.disv_top.txt',
 'hagfors_1.disv_vertices.txt',
 'hagfors_1.ghb',
 'hagfors_1.hds',
 'hagfors_1.ic',
 'hagfors_1.ic_strt.txt',
 'hagfors_1.ims',
 'hagfors_1.lst',
 'hagfors_1.nam',
 'hagfors_1.npf',
 'hagfors_1.npf_icelltype.txt',
 'hagfors_1.npf_k.txt',
 'hagfors_1.npf_k33.txt',
 'hagfors_1.obs',
 'hagfors_1.oc',
 'hagfors_1.rch',
 'hagfors_1.sfr',
 'hagfors_1.sfr.bud',
 'hagfors_1.sfr.obs',
 'hagfors_1.sto',
 'hagfors_1.sto_iconvert.txt',
 'hagfors_1.sto_ss.txt',
 'hagfors_1.sto_sy.txt',
 'hagfors_1.tdis',
 'head.obs.csv',
 'lr_blue.tpl',
 'lr_green.tpl',
 'lr_lu1.tpl',
 'lr_red.tpl',
 'lr_yellow.tpl',
 'mfsim.lst',
 'mfsim.nam',
 'rch_new.ts',
 'sfr_gage.obs.csv',
 'sfr_pred_leak.obs.csv',
 'sfr_reach1_inflow.ts',
 'sfr_reach_evaporation.ts',
 'sfr_reach_rainfall.ts',
 'sfr_rtp_initial.csv',
 'sfr_segfile.dat',
 '

In [5]:
ml_name = 'hagfors_1'

In [6]:
template_ws = os.path.join('..', f'{ml_name}_template')

pf = pyemu.utils.PstFrom(
    original_d=tmp_model_ws,
    new_d=template_ws,
    remove_existing=True,
    longnames=True,
    spatial_reference=None,
    zero_based=False,
    tpl_subfolder='templates',
    start_datetime='1-1-2016'
)

2022-02-02 17:03:51.839668 starting: opening PstFrom.log for logging
2022-02-02 17:03:51.842656 starting PstFrom process
2022-02-02 17:03:51.843654 starting: setting up dirs
2022-02-02 17:03:51.844655 starting: copying original_d '..\temp_pst_from' to new_d '..\hagfors_1_template'
2022-02-02 17:03:51.978292 finished: copying original_d '..\temp_pst_from' to new_d '..\hagfors_1_template' took: 0:00:00.133637
2022-02-02 17:03:51.979620 finished: setting up dirs took: 0:00:00.135966


In [7]:
sim = flopy.mf6.MFSimulation.load(ml_name, 'mf6', os.path.join(bins_pth, 'mf6'), template_ws)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package rch...
    loading package sto...
    loading package oc...
    loading package npf...
    loading package ic...
    loading package sfr...
    loading package obs...
    loading package ghb...
  loading ims package hagfors_1...


### Copy template files into .tpl folder

In [8]:
def move_files(old_pth, new_pth, file):
    from_ = os.path.join(old_pth, file)
    to_ = os.path.join(new_pth, file)
    shutil.move(from_, to_)

In [13]:
for filename in os.listdir(tmp_model_ws):
    if '.tpl' in filename:
        shutil.move(os.path.join(template_ws, filename), os.path.join(template_ws, 'templates', filename))

## 1. Add parameters

Set all data external so that PEST can adjust parameter values during history matching:

In [ ]:
sim.set_all_data_external(True)

In [ ]:
sim.write_simulation()

In [ ]:
gwf = sim.get_model(ml_name)

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('Model grid', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

plt.show()

### 1.1 Generate pilot points for kh, kv, sy and ss

Import (head, stage & flow) measurement locations, so that we can see how the pilot points will be located in relation to them:

In [ ]:
sfr_obs = pd.read_excel(os.path.join(observations_pth, 'obs_flow_and_stage.xlsx')) # contains both stage & flow
head_obs = pd.read_excel(os.path.join(observations_pth, 'obs_head_per_layer.xlsx'))#.drop_duplicates(subset=['POINT_X', 'POINT_Y'])
head_obs['TYPE'] = 'HEAD'

In [ ]:
measurement_points = pd.concat([head_obs, sfr_obs])
measurement_points['NAME'] = measurement_points['NAME'].str.lower()
measurement_points.head()

In [ ]:
hds_meas = pyemu.utils.smp_to_dataframe(os.path.join('..', 'data', 'olproc_input', 'obs-head-all.ssf'))
stg_meas = pyemu.utils.smp_to_dataframe(os.path.join('..', 'data', 'olproc_input', 'obs-stage1.ssf'))
inf_meas = pyemu.utils.smp_to_dataframe(os.path.join('..', 'data', 'olproc_input', 'obs-gage1.ssf'))

In [ ]:
for df in [hds_meas, stg_meas, inf_meas]:
    df['name'] = df['name'].str.lower()

In [ ]:
measurement_points = measurement_points.loc[measurement_points['NAME'].isin(pd.concat([hds_meas, stg_meas, inf_meas]).name.unique())]

#### Generate pilot points using shapely

In [ ]:
ml_boundary = shapefile.Reader(os.path.join(shapefile_pth, 'model_boundary.shp')) # Model boundary shapefile
mlb_shape = np.array(np.rint(ml_boundary.shapeRecords()[0].shape.points)) # Model boundary array

Instantiate a shapely polygon of the model boundary:

In [ ]:
mlb_shapely = Polygon(mlb_shape)

Create prepared polygon of the model boundary:

In [ ]:
mlb_shapely_prep = prep(mlb_shapely)

Construct a rectangular mesh of points:

In [ ]:
xmin, xmax, ymin, ymax = 426900, 427700, 6654650, 6655350 # These are the same coordinates used to construct the base-grid
resolution = 28 # Equal space (in meters) between pilot points
basepoints = []
for lat in np.arange(xmin, xmax, resolution):
    for lon in np.arange(ymin, ymax, resolution):
        basepoints.append(Point((round(lat,4), round(lon,4))))

Use the shapely `contains` (point-in-polygon method) to select points inside the model boundary (increase the number of pps once workflow is confirmed working):

In [ ]:
basepip = [] # Basepoints in polygon
for point in basepoints:
    if mlb_shapely_prep.contains(point):
        basepip.append(point)
print(f'Number of points per layer: {len(basepip)}') # We need to extend it into three dimensions

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=100)
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('Coarse pilot point locations', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

x = np.array([i.coords[0] for i in basepip])[:,0]
y = np.array([i.coords[0] for i in basepip])[:,1]
plt.scatter(x, y, s=5, c='black', alpha=0.5, label='PILOT POINT')

for category in measurement_points['TYPE'].unique():
    x = measurement_points.loc[measurement_points['TYPE'] == category]['POINT_X']
    y = measurement_points.loc[measurement_points['TYPE'] == category]['POINT_Y']
    ax.scatter(x, y, s=20, alpha=0.6, label=category)
    
plt.legend()
plt.show()

These points now need to be assigned a z-value, since we are going to use 3D-pilot points. To do this, we will need to intersect the model grid and retrieve the z-values of each layer, so that a copy of these pps can be positioned in each of the three layers:

In [ ]:
ix = GridIntersect(gwf.modelgrid)

In [ ]:
pp_intersect = {
    'cellids': [],
    'vertices': [],
    'ixshapes': [],
}
for point in basepip:
    pp_intersect['cellids'].append(ix.intersect(point).cellids[0])
    pp_intersect['vertices'].append(ix.intersect(point).vertices[0])
    pp_intersect['ixshapes'].append(ix.intersect(point).ixshapes[0])

In [ ]:
pp_intersect = pd.DataFrame(pp_intersect)
display(pp_intersect)

In [ ]:
def get_grid_elevation(elevation_array, index_array):
    '''
    returns list of elevations
    '''
    
    elevations = list(elevation_array)
    indices = list(index_array)
    
    return [elevation_array[i] for i in index_array]

In [ ]:
pp_intersect['top'] = get_grid_elevation(elevation_array = gwf.modelgrid.top, index_array=pp_intersect['cellids'])
pp_intersect['botm_1'] = get_grid_elevation(gwf.modelgrid.botm[0], pp_intersect['cellids'])
pp_intersect['botm_2'] = get_grid_elevation(gwf.modelgrid.botm[1], pp_intersect['cellids'])
pp_intersect['botm_3'] = get_grid_elevation(gwf.modelgrid.botm[2], pp_intersect['cellids'])

Create pps in-between the layer boundaries (i.e. vertically centered in the cells):

In [ ]:
pp_intersect['pps_l1'] = (pp_intersect['top'] + pp_intersect['botm_1']) / 2
pp_intersect['pps_l2'] = (pp_intersect['botm_1'] + pp_intersect['botm_2']) / 2
pp_intersect['pps_l3'] = (pp_intersect['botm_2'] + pp_intersect['botm_3']) / 2
display(pp_intersect)

In [ ]:
pp_intersect['x'] = np.array([i.coords[0] for i in basepip])[:,0]
pp_intersect['y'] = np.array([i.coords[0] for i in basepip])[:,1]
pp_intersect = pp_intersect[['x', 'y', 'pps_l1', 'pps_l2', 'pps_l3']]

Convert the dataframe into a 3d pilot point file format:

In [ ]:
pp_coarse3d = pd.melt(pp_intersect, id_vars=['x', 'y'], value_vars=['pps_l1', 'pps_l2', 'pps_l3'])

In [ ]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure(figsize=(6,6))
ax = Axes3D(fig)
ax.set_box_aspect([2,2,1])

x, y = pp_coarse3d.x.values, pp_coarse3d.y.values

ax.scatter(x, y, pp_coarse3d.value.values)
plt.title('pps in 3d-space')
pyplot.show()

In [ ]:
pp_coarse3d['name'] = [f'ppc{i:04d}' for i in pp_coarse3d.index.values] # ppc = pilot point coarse.

In [ ]:
pp_coarse3d[['zone','val']] = 1, 1 #redundant?
pp_coarse3d['z'] = pp_coarse3d['value']

In [ ]:
pp_coarse3d['layer'] = [int(i[-1]) for i in pp_coarse3d['variable']]

In [ ]:
pp_coarse3d = pp_coarse3d[['name', 'x', 'y', 'z', 'zone', 'val', 'layer']]

In [ ]:
display(pp_coarse3d)

Covariance matrix:

In [ ]:
from lumpyrem import run
#MKPPSTAT
# mkppstat requires no headers in ppoint file ...sigh...
pp_coarse3d.to_csv(os.path.join(template_ws, 'mkppoints3d_coarse.dat'),
                       header=None, index=False, sep='\t')

In [ ]:
# Input for MKPPSTAT
# A (a)-factor of 1.5 is often reasonable. from tutorial 
npoints_h, npoints_v = 10, 10 # np-horizontal, np-vertical
a_h, a_v = 1.2, 1.2 # a-horizontal, a-vertical
# run MKPPSTAT
run.run_process(
    'mkppstat3d',
    path=template_ws,
    commands=['mkppoints3d_coarse.dat', npoints_h, a_h, npoints_v, a_v, 'ppstat3d_coarse.dat']
)

In [ ]:
# run PPCOV3D_SVA - pilot point covariance 3d - spatially varying anisotropy
run.run_process(
    'ppcov3d_sva',
    path=template_ws,
    commands=['ppstat3d_coarse.dat', 'y', 1, 'x',  'cov3d_coarse.mat', '']
)

In [ ]:
covmat_coarse = pyemu.Cov.from_ascii(os.path.join(template_ws, "cov3d_coarse.mat"))

# This covaraince matrix can now be used as the base for all pilot point parameters. 
# Note that in this case the variance is 1, so it is easy to scale to a parameters prior varaince
# Depending on how you setup the scrpt, variance can be assigned at various stages (i.e. when running PPCOV_SVA, or by manipulating the matrix later)
# Note that parameter names (headers and row names) come from the parameter name sin the ppoint file. These can have a prefix added by PPCOV_SVA, or changed in the dataframe. The latter is more versatile.
covmat_coarse.to_dataframe().head()

In [ ]:
pf.add_parameters(
    filenames=f'{ml_name}.npf_k_layer1.txt',
    par_type='pilotpoints',
    par_name_base="hk_layer_1",
    pargp="hk_layer_1",
    #zone_array=ib,
    upper_bound=864.0,
    lower_bound=0.864,
    ult_ubound=1200.0,
    ult_lbound=0.00001,
    transform='log',
    #geostruct=covmat_coarse, AttributeError: 'Cov' object has no attribute 'sill'
    pp_space=pp_coarse3d.loc[pp_coarse3d['layer'] == 1]
)

### General Head Boundaries (GHB)

In [ ]:
ghbpp = pd.read_csv(os.path.join(tmp_model_ws, 'ghbpp.dat'), sep='\t')
ghbpp.head()

In [ ]:
ghbpp['zone'] = 1

In [ ]:
ghbpp = ghbpp[['ppid', 'x', 'y', 'zone', 'cond', 'bnames']]

In [ ]:
ghbpp.rename(columns={'cond': 'parval1'}, inplace=True)

In [ ]:
ghbppred = ghbpp.loc[ghbpp['bnames'] == 'red'][['ppid', 'x', 'y', 'zone', 'parval1']]

In [ ]:
ghbppred

In [ ]:
pf.add_parameters(
    filenames=f'{ml_name}.ghb_stress_period_data_1.txt', 
    par_name_base='ghbr_rhk',
    pargp='ghb_hk',
    index_cols=[0,1], 
    use_cols=[3],
    upper_bound=10.,
    lower_bound=0.1,
    ult_ubound=100.,
    ult_lbound=0.01,
    par_type='pilotpoints',
    pp_space=ghbppred
)

### Stream Flow Routing (SFR)

In [ ]:
# add grid-scale parameters for SFR reach conductance.  
# Use layer, row, col and reach number in the 
# parameter names
pf.add_parameters(
    filenames=f'{ml_name}.sfr_packagedata.txt', 
    par_name_base='sfr_rhk',
    pargp='sfr_rhk',
    index_cols=[0,1,2], 
    use_cols=[9],
    upper_bound=10.,
    lower_bound=0.1,
    ult_ubound=100.,
    ult_lbound=0.01,
    par_type='grid'
)

In [ ]:
pp_coarse3d.loc[pp_coarse3d['layer'] == 1]

## 2. Add observations

### 2.1 Heads

#### Computed head

- Is there a point of having "hds_usecol:" in the obsnme and obgnme or can this be removed safely?

In [ ]:
df = pd.read_csv(os.path.join(template_ws, 'head.obs.csv'),index_col=0)
df.head()

In [ ]:
sp_start = pd.to_datetime('2016-01-01') + pd.to_timedelta(df.index.values,unit='d')

In [ ]:
hds_file = os.path.join(tmp_model_ws, 'head.obs.csv')

hds_df = pf.add_observations(
    hds_file,
    insfile='head.csv.ins',
    index_cols='time',
    use_cols=list(df.columns.values),
    prefix='hds'
)

In [ ]:
hds_df

Set weights to zero:

In [ ]:
hds_df.loc[:,'weight'] = 0.0

In [ ]:
hds_df.head()

#### Measured head
The plan is to change the `obsval` of `hds_df` with real, measured data. Let's load the field measured data:

In [ ]:
hds_meas = pyemu.utils.smp_to_dataframe(os.path.join('..', 'data', 'olproc_input', 'obs-head-all.ssf'))
hds_meas['name'] = hds_meas['name'].str.lower()
hds_meas

#### Plot measured heads

In [ ]:
for site in hds_meas['name'].unique():
    
    site_obs_data = hds_meas.loc[hds_meas['name'] == site]
    
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    
    ax.plot(site_obs_data.datetime,site_obs_data.value)
    ax.set_title(site)

plt.show()

These datasets will now be sampled for use in history-matching purposes. Sampling can be done in different ways. Let's explore two options based on a notebook developed by Jeremy White (https://github.com/jtwhite79/decision_support_analysis_notebooks/blob/main/notebooks/process_obs_and_set_weights.ipynb) and visualize the options together with the raw data.

#### Option 1: sampling the nearest data

In [ ]:
hds_meas_nearest, hds_meas_lowpass = [], []

In [ ]:
for site in hds_meas['name'].unique():
    site_obs_data = hds_meas.loc[hds_meas['name'] == site].copy()
    site_obs_data.index = site_obs_data.datetime
    site_obs_data = site_obs_data.reindex(sp_start,method='nearest')
    site_obs_data['datetime'] = site_obs_data.index
    site_obs_data.sort_values(by='datetime', inplace=True)
    hds_meas_nearest.append(site_obs_data)
hds_meas_nearest = pd.concat(hds_meas_nearest,axis=0,ignore_index=True)
hds_meas_nearest.shape

This dataframe now contain alot of duplicate values (as is apparent below). Let's clean this up.

In [ ]:
hds_meas_nearest

In [ ]:
hds_meas_nearest = hds_meas_nearest.drop_duplicates('value').reset_index(drop=True)

#### Option 2: using a lowpass filter

In [ ]:
for site in hds_meas['name'].unique():
    site_obs_data = hds_meas.loc[hds_meas['name'] == site].copy()
    site_obs_data.index = site_obs_data.datetime
    
    sm = site_obs_data.value.rolling(window=60,center=True,min_periods=1).mean()
    sm_site_obs_data = sm.reindex(sp_start,method='nearest')

    df = pd.DataFrame(sm_site_obs_data)
    df['datetime'], df['name'] = df.index, site
    df = df[['datetime', 'name', 'value']]
    df.sort_values(by="datetime",inplace=True)
    hds_meas_lowpass.append(df)
hds_meas_lowpass = pd.concat(hds_meas_lowpass,axis=0,ignore_index=True).drop_duplicates('value').reset_index(drop=True)
hds_meas_lowpass.shape

Ensure that sampled data is only collected from time-periods of when actual measurements were collected:

In [ ]:
obs_sites = hds_meas['name'].unique().copy()

In [ ]:
def add_timedeltas(df, model_start_time):
    df['timedelta'] = [(dt - pd.to_datetime(model_start_time)).total_seconds() / 86400 for dt in df.datetime]

In [ ]:
for df in [hds_meas, hds_meas_nearest, hds_meas_lowpass]:
    add_timedeltas(df, '2016-01-01')

In [ ]:
def match_timedeltas(measured_df, timeinterp_df):
    '''
    This function compares timedeltas of the time-interpolated values with that of the field measurements.
    
    If a time-interpolated value is within +- 2 days of a field measurement,
    this value is accepted, else the row is dropped.
    '''
    timedelta_match = []
    
    for i, r in timeinterp_df.iterrows():

        results = []

        for j in measured_df.timedelta:
            result = j + 2 >= r.timedelta >= j - 2 # determine whether there is a match between datasets
            results.append(result)

        if any(results) == True:
            timedelta_match.append(True)
        else:
            timedelta_match.append(False)
            
    return timedelta_match

In [ ]:
for df in [hds_meas_nearest, hds_meas_lowpass]:
    df['td_match'] = False
    
    # Loop over each site
    for site in hds_meas.name.unique():
        meas_df = hds_meas.loc[hds_meas['name'] == site]
        df_sub = df.loc[df['name'] == site]
        
        df.loc[df['name'] == site, 'td_match'] = match_timedeltas(meas_df, df_sub)

In [ ]:
hds_meas_nearest = hds_meas_nearest.loc[hds_meas_nearest['td_match'] == True].reset_index(drop=True)
hds_meas_lowpass = hds_meas_lowpass.loc[hds_meas_lowpass['td_match'] == True].reset_index(drop=True)

#### Visualize the two sampling options. `Nearest data` samples are shown in red. `Lowpass data` samples are shown in green:

In [ ]:
for site in obs_sites:
    
    site_obs_data = hds_meas.loc[hds_meas['name'] == site]
    site_obs_data_nearest = hds_meas_nearest.loc[hds_meas_nearest['name'] == site]
    site_obs_data_lowpass = hds_meas_lowpass.loc[hds_meas_lowpass['name'] == site]
    
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    
    ax.plot(site_obs_data.datetime,site_obs_data.value)
    ax.plot(site_obs_data_nearest.datetime,site_obs_data_nearest.value,'r-',lw=1,marker='.',ms=10)
    ax.scatter(site_obs_data_nearest.datetime,site_obs_data_nearest.value, color='r')

    ax.plot(site_obs_data_lowpass.datetime, site_obs_data_lowpass.value,'g-',lw=1,marker='.',ms=10)
    ax.scatter(site_obs_data_lowpass.datetime, site_obs_data_lowpass.value, color='g')

plt.show()

The lowpass sampling method seem to capture the trend of the data better than using the nearest option. Because of this, the lowpass interpolated measurements will be used in the history-matching process.

#### Replace model `obsval` with `lowpass` interpolated measurements
A reminder of how the PEST observations are setup:

In [ ]:
hds_df.head()

Now, the mission is to assign a weight > 0 for the observations where we have time-interpolated (using a lowpass filter) measurements. Recall that the time-interpolated measurements are located in the `hds_meas_lowpass` DataFrame. The values in this dataframe must now be matched with the corresponding values of `hds_df`.

As we can see, we have a `datetime` column. This information must be converted into model time so that values can be matched:

In [ ]:
hds_meas_lowpass.iloc[:]

In [ ]:
def build_matching_obs_columns(df, col_prefix):
    
    # Build partial obsnme column (for matching against PEST observations dataframe)
    df['matchcol'] = [f'{col_prefix}_usecol:{i}_time:{j}' for i, j in zip(df['name'], df['timedelta'])]
    df['matchcol'] = [i[:i.index('.') + 2] for i in df['matchcol']]
    
    df['obsgnme'] = [f'{col_prefix}_usecol:{i}' for i in df['name']]
    
    df['weight'] = 1.0
    
    df['obsval'] = df['value']

In [ ]:
build_matching_obs_columns(hds_meas_lowpass, 'hds')

`obsnme` in the `hds_meas_lowpass` dataframe will now be used to find matching `obsnme` rows in `hds_df` and then replace `obsval` and `weight` with time-interpolated field measurements:

In [ ]:
hds_meas_lowpass.head()

Add a temporary `matchcol` column so that `obsval` can be changed to time-interpolated measured counterparts:

In [ ]:
hds_df['matchcol'] = [i[:i.index('.') + 2] for i in hds_df['obsnme']]

Assign the time-interpolated measurements to the `obsval` and give these measurements a positive weight for the history-matching process:

In [ ]:
for i,r in hds_meas_lowpass.iterrows():

    hds_df.loc[hds_df.matchcol == r.matchcol, 'weight'] = 1.0
    hds_df.loc[hds_df.matchcol == r.matchcol, 'obsval'] = r.value

In [ ]:
hds_df.loc[hds_df['weight'] == 1.0]

Remove the temporary `matchcol` column from the history-matching dataset:

In [ ]:
hds_df.drop('matchcol', axis=1, inplace=True)

**Setting up history-matching for heads completed.**

### 2.2 Head differences

Create the csv-file from measured head that will be read

In [ ]:
df = pd.read_csv(os.path.join(template_ws, 'head.obs.csv'),index_col=0)
df.head()

In [ ]:
for i,r in df.iterrows():
    print(r.name)

In [ ]:
hds_meas_lowpass_td = hds_meas_lowpass.copy() # temporal difference

In [ ]:
hds_meas_lowpass_td

### 2.3 Stage 

Read computed stage:

In [ ]:
df = pd.read_csv(os.path.join(template_ws, 'sfr_stage.obs.csv'),index_col=0)
df.head()

In [ ]:
stg_file = os.path.join(template_ws, 'sfr_stage.obs.csv')

stg_df = pf.add_observations(
    stg_file,
    insfile='stage.csv.ins',
    index_cols='time',
    use_cols=list(df.columns.values),
    prefix='stg'
)

In [ ]:
stg_df.loc[:,'weight'] = 0.0

In [ ]:
stg_df.head()

In [ ]:
stg_meas = pyemu.utils.smp_to_dataframe(os.path.join('..', 'data', 'olproc_input', 'obs-stage1.ssf'))
stg_meas['name'] = stg_meas['name'].str.lower()
stg_meas

In [ ]:
for site in stg_meas['name'].unique():
    
    site_obs_data = stg_meas.loc[stg_meas['name'] == site]
    
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    
    ax.plot(site_obs_data.datetime,site_obs_data.value)
    ax.set_title(site)

plt.show()

In [ ]:
stg_meas_nearest, stg_meas_lowpass = [], []

In [ ]:
for site in stg_meas['name'].unique():
    site_obs_data = stg_meas.loc[stg_meas['name'] == site].copy()
    site_obs_data.index = site_obs_data.datetime
    site_obs_data = site_obs_data.reindex(sp_start,method='nearest')
    site_obs_data['datetime'] = site_obs_data.index
    site_obs_data.sort_values(by='datetime', inplace=True)
    stg_meas_nearest.append(site_obs_data)
stg_meas_nearest = pd.concat(stg_meas_nearest,axis=0,ignore_index=True).drop_duplicates('value').reset_index(drop=True)
stg_meas_nearest.shape

In [ ]:
for site in stg_meas['name'].unique():
    site_obs_data = stg_meas.loc[stg_meas['name'] == site].copy()
    site_obs_data.index = site_obs_data.datetime
    
    sm = site_obs_data.value.rolling(window=30,center=True,min_periods=1).mean() # test window of 30 inst of 60
    sm_site_obs_data = sm.reindex(sp_start,method='nearest')

    df = pd.DataFrame(sm_site_obs_data)
    df['datetime'], df['name'] = df.index, site
    df = df[['datetime', 'name', 'value']]
    df.sort_values(by="datetime",inplace=True)
    stg_meas_lowpass.append(df)
stg_meas_lowpass = pd.concat(stg_meas_lowpass,axis=0,ignore_index=True).drop_duplicates('value').reset_index(drop=True)
stg_meas_lowpass.shape

In [ ]:
obs_sites = stg_meas['name'].unique().copy()

In [ ]:
for df in [stg_meas, stg_meas_nearest, stg_meas_lowpass]:
    add_timedeltas(df, '2016-01-01')

In [ ]:
for df in [stg_meas_nearest, stg_meas_lowpass]:
    df['td_match'] = False
    
    # Loop over each site
    for site in stg_meas.name.unique():
        meas_df = stg_meas.loc[stg_meas['name'] == site]
        df_sub = df.loc[df['name'] == site]
        
        df.loc[df['name'] == site, 'td_match'] = match_timedeltas(meas_df, df_sub)

In [ ]:
stg_meas_nearest = stg_meas_nearest.loc[stg_meas_nearest['td_match'] == True].reset_index(drop=True)
stg_meas_lowpass = stg_meas_lowpass.loc[stg_meas_lowpass['td_match'] == True].reset_index(drop=True)

In [ ]:
for site in obs_sites:
    
    site_obs_data = stg_meas.loc[stg_meas['name'] == site]
    site_obs_data_nearest = stg_meas_nearest.loc[stg_meas_nearest['name'] == site]
    site_obs_data_lowpass = stg_meas_lowpass.loc[stg_meas_lowpass['name'] == site]
    
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    
    ax.plot(site_obs_data.datetime,site_obs_data.value)
    ax.plot(site_obs_data_nearest.datetime,site_obs_data_nearest.value,'r-',lw=1,marker='.',ms=10)
    ax.scatter(site_obs_data_nearest.datetime,site_obs_data_nearest.value, color='r')

    ax.plot(site_obs_data_lowpass.datetime, site_obs_data_lowpass.value,'g-',lw=1,marker='.',ms=10)
    ax.scatter(site_obs_data_lowpass.datetime, site_obs_data_lowpass.value, color='g')

plt.show()

In [ ]:
stg_df

In [ ]:
stg_meas_lowpass

In [ ]:
build_matching_obs_columns(stg_meas_lowpass, 'stg')

In [ ]:
stg_meas_lowpass.head()

In [ ]:
stg_df['matchcol'] = [i[:i.index('.') + 2] for i in stg_df['obsnme']]

In [ ]:
for i,r in stg_meas_lowpass.iterrows():

    stg_df.loc[stg_df.matchcol == r.matchcol, 'weight'] = 1.0
    stg_df.loc[stg_df.matchcol == r.matchcol, 'obsval'] = r.value

In [ ]:
stg_df.loc[stg_df['weight'] == 1.0]

In [ ]:
stg_df.drop('matchcol', axis=1, inplace=True)

**Setting up history-matching for stage completed.**

### 2.4 Stage differences

### 2.5 Inflow (Stream gage)

In [ ]:
df = pd.read_csv(os.path.join(template_ws, 'sfr_gage.obs.csv'),index_col=0)
df.head()

In [ ]:
inf_file = os.path.join(tmp_model_ws, 'sfr_gage.obs.csv')

inf_df = pf.add_observations(
    inf_file,
    insfile='sfr_gage.csv.ins',
    index_cols='time',
    use_cols=list(df.columns.values),
    prefix='inf'
)

In [ ]:
inf_df

In [ ]:
inf_df.loc[:,'weight'] = 0.0

In [ ]:
inf_df.head()

In [ ]:
inf_meas = pyemu.utils.smp_to_dataframe(os.path.join('..', 'data', 'olproc_input', 'obs-gage1.ssf'))
inf_meas['name'] = inf_meas['name'].str.lower()
inf_meas

In [ ]:
for site in inf_meas['name'].unique():
    
    site_obs_data = inf_meas.loc[inf_meas['name'] == site]
    
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    
    ax.plot(site_obs_data.datetime,site_obs_data.value)
    ax.set_title(site)

plt.show()

In [ ]:
inf_meas_nearest, inf_meas_lowpass = [], []

In [ ]:
for site in inf_meas['name'].unique():
    site_obs_data = inf_meas.loc[inf_meas['name'] == site].copy()
    site_obs_data.index = site_obs_data.datetime
    site_obs_data = site_obs_data.reindex(sp_start,method='nearest')
    site_obs_data['datetime'] = site_obs_data.index
    site_obs_data.sort_values(by='datetime', inplace=True)
    inf_meas_nearest.append(site_obs_data)
inf_meas_nearest = pd.concat(inf_meas_nearest,axis=0,ignore_index=True).drop_duplicates('value').reset_index(drop=True)
inf_meas_nearest.shape

In [ ]:
for site in inf_meas['name'].unique():
    site_obs_data = inf_meas.loc[inf_meas['name'] == site].copy()
    site_obs_data.index = site_obs_data.datetime
    
    sm = site_obs_data.value.rolling(window=30,center=True,min_periods=1).mean() #30 window
    sm_site_obs_data = sm.reindex(sp_start,method='nearest')

    df = pd.DataFrame(sm_site_obs_data)
    df['datetime'], df['name'] = df.index, site
    df = df[['datetime', 'name', 'value']]
    df.sort_values(by='datetime', inplace=True)
    inf_meas_lowpass.append(df)
inf_meas_lowpass = pd.concat(inf_meas_lowpass,axis=0,ignore_index=True).drop_duplicates('value').reset_index(drop=True)
inf_meas_lowpass.shape

In [ ]:
obs_sites = inf_meas['name'].unique().copy()

In [ ]:
for df in [inf_meas, inf_meas_nearest, inf_meas_lowpass]:
    add_timedeltas(df, '2016-01-01')

In [ ]:
for df in [inf_meas_nearest, inf_meas_lowpass]:
    df['td_match'] = False
    
    # Loop over each site
    for site in inf_meas.name.unique():
        meas_df = inf_meas.loc[inf_meas['name'] == site]
        df_sub = df.loc[df['name'] == site]
        
        df.loc[df['name'] == site, 'td_match'] = match_timedeltas(meas_df, df_sub)

In [ ]:
inf_meas_nearest = inf_meas_nearest.loc[inf_meas_nearest['td_match'] == True].reset_index(drop=True)
inf_meas_lowpass = inf_meas_lowpass.loc[inf_meas_lowpass['td_match'] == True].reset_index(drop=True)

In [ ]:
for site in obs_sites:
    
    site_obs_data = inf_meas.loc[inf_meas['name'] == site]
    site_obs_data_nearest = inf_meas_nearest.loc[inf_meas_nearest['name'] == site]
    site_obs_data_lowpass = inf_meas_lowpass.loc[inf_meas_lowpass['name'] == site]
    
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    
    ax.plot(site_obs_data.datetime,site_obs_data.value)
    ax.plot(site_obs_data_nearest.datetime,site_obs_data_nearest.value,'r-',lw=1,marker='.',ms=10)
    ax.scatter(site_obs_data_nearest.datetime,site_obs_data_nearest.value, color='r')

    ax.plot(site_obs_data_lowpass.datetime, site_obs_data_lowpass.value,'g-',lw=1,marker='.',ms=10)
    ax.scatter(site_obs_data_lowpass.datetime, site_obs_data_lowpass.value, color='g')

plt.show()

In [ ]:
inf_df.head()

In [ ]:
inf_meas_lowpass

In [ ]:
build_matching_obs_columns(inf_meas_lowpass, 'inf')

In [ ]:
inf_meas_lowpass.head()

In [ ]:
inf_df['matchcol'] = [i[:i.index('.') + 2] for i in inf_df['obsnme']]

In [ ]:
for i,r in inf_meas_lowpass.iterrows():

    inf_df.loc[inf_df.matchcol == r.matchcol, 'weight'] = 1.0
    inf_df.loc[inf_df.matchcol == r.matchcol, 'obsval'] = r.value

In [ ]:
inf_df.loc[inf_df['weight'] == 1.0]

In [ ]:
inf_df.drop('matchcol', axis=1, inplace=True)

In [ ]:
inf_df

In [ ]:
sum(inf_df['weight'])

### 2.6 Inflow differences

### 2.7 Head-Stage differences

This is a customized measurement type

Start by creating the synthetic dataset:

In [ ]:
hds_comp = pd.read_csv(os.path.join(template_ws, 'head.obs.csv'),index_col=0)
hds_comp.head()

In [ ]:
stg_comp = pd.read_csv(os.path.join(template_ws, 'sfr_stage.obs.csv'),index_col=0)
stg_comp.head()

In [ ]:
columns = [
    'NI15-O1', 'NI15-O1-STG',
    'NI15-O44', 'NI15-O44-STG',
    'NI15-O46', 'NI15-O46-STG',
    'NI15-O47', 'NI15-O47-STG',
    'NI15-O48', 'NI15-O48-STG'
]

In [ ]:
dh = pd.concat([hds_comp, stg_comp], axis=1)[columns]

In [ ]:
for i, j in zip(columns[::2], columns[1::2]):
    dh[f'dh{i}'] = dh[i] - dh[j]

In [ ]:
dh = dh[[f'dh{i}' for i in columns[::2]]]

In [ ]:
dh.head()

In [ ]:
dh.to_csv(os.path.join(template_ws, 'dh_orbacken.obs.csv'))

In [ ]:
dho_file = os.path.join(template_ws, 'dh_orbacken.obs.csv')

dho_df = pf.add_observations( #delta h örbäcken (head stage difference at 5 points)
    dho_file,
    insfile='dh_orbacken.csv.ins',
    index_cols='time',
    use_cols=list(dh.columns.values),
    prefix='dho'
)

In [ ]:
dho_df['weight'] = 0.0

In [ ]:
dho_df.head()

## End of notebook

---

Setup directory structure:

In [ ]:
model_root = os.path.join('..', 'temp_ml_param')

runmodel_ws = os.path.join(model_root, 'runmodel')
model_ws = os.path.join(runmodel_ws, 'model')
preproc_ws = os.path.join(runmodel_ws, 'preproc')
lumprem_ws = os.path.join(runmodel_ws, 'lumprem')

pest_ws = os.path.join(model_root, 'pest')
template_ws = os.path.join(pest_ws, 'template')
instruction_ws = os.path.join(pest_ws, 'instruction')

In [ ]:
dir_lvl1 = [runmodel_ws, pest_ws]
dir_lvl2 = [model_ws, preproc_ws, lumprem_ws, template_ws, instruction_ws]

In [ ]:
if os.path.exists(model_root):
    shutil.rmtree(model_root)

os.mkdir(model_root)

In [ ]:
for pth in dir_lvl1:
    os.mkdir(pth)
for pth in dir_lvl2:
    os.mkdir(pth)

In [ ]:
shutil.copytree(org_model_ws,model_ws, dirs_exist_ok=True)
os.listdir(model_ws)

Move all the LUMPREM template files to `template_ws` and SEGLIST files to `preproc_ws` directories:

In [ ]:
def move_files(old_pth, new_pth, file):
    from_ = os.path.join(old_pth, file)
    to_ = os.path.join(new_pth, file)
    shutil.move(from_, to_)

In [ ]:
lr_tpls = [i for i in os.listdir(model_ws) if '.tpl' in i]

In [ ]:
for file in lr_tpls:
    move_files(model_ws, template_ws, file)

In [ ]:
move_files(model_ws, preproc_ws, 'sfr_segfile.dat')
move_files(model_ws, preproc_ws, 'ghbpp.dat')

Move all the LUMPREM input files to `lumprem_ws`:

In [ ]:
lr_prep_pth = os.path.join('..', 'temp_lr_prep')

In [ ]:
os.listdir(lr_prep_pth)

In [ ]:
for file in os.listdir(lr_prep_pth):
    shutil.copyfile(os.path.join(lr_prep_pth, file), os.path.join(lumprem_ws, file))

In [ ]:
os.listdir(lumprem_ws)

START

In [ ]:
ml_name = 'hagfors_1'

In [ ]:
sim = flopy.mf6.MFSimulation.load(ml_name, 'mf6', os.path.join(bins_pth, 'mf6'), model_ws)

Set all data external so that PEST can adjust parameter values during history matching:

In [ ]:
sim.set_all_data_external(True)

In [ ]:
sim.write_simulation()

In [ ]:
gwf = sim.get_model(ml_name)

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('Model grid', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

plt.show()

## Generate pilot points
3D-pilot points will be generated for the following areal properties:
- SY & SS (perhaps SS could be skipped?)
- Porosity and possibly diffusion/dispersion and any other parameter needed to represent transport
- Kh and Kz

In addition, pilot points will be generated along the following linear features:
- Streambed hydraulic conductivity along Creek Örbäcken (SFR package)
- GHB conductance

### 3D pilot points
Because we want to do a data-worth analysis, we should consider the total amount of pilot points to be employed to find a suitable compromise between adjustable parameters and model run-time.

For this reason, we will create different sets of pilot points for each parameter type (in order to not use an exessive amount of pps).

Sy, SS and possibly porosity (along with other parameters that govern transport) could be parameterized using a coarser pp-spacing, so let's start with that.

However, before we start, let's import the observation locations, so that we can see how the pilot points will be located in relation to them:

In [ ]:
sfr_obs = pd.read_excel(os.path.join(observations_pth, 'obs_flow_and_stage.xlsx'))
head_obs = pd.read_excel(os.path.join(observations_pth, 'obs_head_per_layer.xlsx')).drop_duplicates(subset=['POINT_X', 'POINT_Y'])
head_obs['TYPE'] = 'HEAD'

In [ ]:
obs_points = pd.concat([head_obs, sfr_obs])
display(obs_points)

#### Coarse 3D pilot points
Let's create the coarse pilot point distribution:

In [ ]:
ml_boundary = shapefile.Reader(os.path.join(shapefile_pth, 'model_boundary.shp')) # Model boundary shapefile
mlb_shape = np.array(np.rint(ml_boundary.shapeRecords()[0].shape.points)) # Model boundary array

Instantiate a shapely polygon of the model boundary:

In [ ]:
mlb_shapely = Polygon(mlb_shape)

Create prepared polygon of the model boundary:

In [ ]:
#mlb_shapely_prep = prep(mlb_shapely.buffer(50)) buffer cause an issue because the grid can't be sampled outside the model boundary
mlb_shapely_prep = prep(mlb_shapely)

Construct a rectangular mesh of points:

In [ ]:
xmin, xmax, ymin, ymax = 426900, 427700, 6654650, 6655350 # These are the same coordinates used to construct the base-grid
resolution = 30 # Equal space (in meters) between pilot points
basepoints = []
for lat in np.arange(xmin, xmax, resolution):
    for lon in np.arange(ymin, ymax, resolution):
        basepoints.append(Point((round(lat,4), round(lon,4))))

Use the shapely `contains` (point-in-polygon method) to select points inside the model boundary (increase the number of pps once workflow is confirmed working):

In [ ]:
basepip = [] # Basepoints in polygon
for point in basepoints:
    if mlb_shapely_prep.contains(point):
        basepip.append(point)
print(f'Number of points per layer: {len(basepip)}') # We need to extend it into three dimensions

Plot the position of the pilot points on top of the model grid:

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('Coarse pilot point locations', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

x = np.array([i.coords[0] for i in basepip])[:,0]
y = np.array([i.coords[0] for i in basepip])[:,1]
plt.scatter(x, y, s=5, c='black', alpha=0.5, label='PILOT POINT')

for category in obs_points['TYPE'].unique():
    x = obs_points.loc[obs_points['TYPE'] == category]['POINT_X']
    y = obs_points.loc[obs_points['TYPE'] == category]['POINT_Y']
    ax.scatter(x, y, s=20, alpha=0.6, label=category)
    
plt.legend()
plt.show()

These points now need to be assigned a z-value, since we are going to use 3D-pilot points. To do this, we will need to intersect the model grid and retrieve the z-values of each layer, so that a copy of these pps can be positioned in each of the three layers:

In [ ]:
ix = GridIntersect(gwf.modelgrid)

In [ ]:
pp_intersect = {
    'cellids': [],
    'vertices': [],
    'ixshapes': [],
}
for point in basepip:
    pp_intersect['cellids'].append(ix.intersect(point).cellids[0])
    pp_intersect['vertices'].append(ix.intersect(point).vertices[0])
    pp_intersect['ixshapes'].append(ix.intersect(point).ixshapes[0])

In [ ]:
pp_intersect = pd.DataFrame(pp_intersect)
display(pp_intersect)

We can see that the order is respected:

In [ ]:
[i.coords[0] for i in basepip][:5]

Let's use the cellids to sample pilot point elevations (**this takes about 2 minutes on my laptop for 295 cells** and could/should probably be speed up somehow - considering it has to be done for top, botm1, botm2 and botm3):

In [ ]:
def get_grid_elevation(elevation_array, index_array):
    '''
    returns list of elevations
    '''
    
    elevations = list(elevation_array)
    indices = list(index_array)
    
    return [elevation_array[i] for i in index_array]

In [ ]:
pp_intersect['top'] = get_grid_elevation(elevation_array = gwf.modelgrid.top, index_array=pp_intersect['cellids'])

In [ ]:
pp_intersect['botm_1'] = get_grid_elevation(gwf.modelgrid.botm[0], pp_intersect['cellids'])
pp_intersect['botm_2'] = get_grid_elevation(gwf.modelgrid.botm[1], pp_intersect['cellids'])
pp_intersect['botm_3'] = get_grid_elevation(gwf.modelgrid.botm[2], pp_intersect['cellids'])

In [ ]:
pp_intersect

Create pps in-between the layer boundaries (i.e. vertically centered in the cells):

In [ ]:
pp_intersect['pps_l1'] = (pp_intersect['top'] + pp_intersect['botm_1']) / 2

In [ ]:
pp_intersect['pps_l2'] = (pp_intersect['botm_1'] + pp_intersect['botm_2']) / 2

In [ ]:
pp_intersect['pps_l3'] = (pp_intersect['botm_2'] + pp_intersect['botm_3']) / 2

In [ ]:
display(pp_intersect)

Add x, y and clean up df:

In [ ]:
pp_intersect['x'] = np.array([i.coords[0] for i in basepip])[:,0]
pp_intersect['y'] = np.array([i.coords[0] for i in basepip])[:,1]

In [ ]:
pp_intersect = pp_intersect[['x', 'y', 'pps_l1', 'pps_l2', 'pps_l3']]

In [ ]:
pp_intersect

Visualize

In [ ]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure(figsize=(6,6))
ax = Axes3D(fig)
ax.set_box_aspect([2,2,1])

x, y = pp_intersect.x.values, pp_intersect.y.values

ax.scatter(x, y, pp_intersect.pps_l1.values)
ax.scatter(x, y, pp_intersect.pps_l2.values)
ax.scatter(x, y, pp_intersect.pps_l3.values)
plt.title('pps in 3d-space')
pyplot.show()

Convert the dataframe into a 3d pilot point file format:

In [ ]:
pp_coarse3d = pd.melt(pp_intersect, id_vars=['x', 'y'], value_vars=['pps_l1', 'pps_l2', 'pps_l3'])

In [ ]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure(figsize=(6,6))
ax = Axes3D(fig)
ax.set_box_aspect([2,2,1])

x, y = pp_coarse3d.x.values, pp_coarse3d.y.values

ax.scatter(x, y, pp_coarse3d.value.values)
plt.title('pps in 3d-space')
pyplot.show()

Insert columns:

In [ ]:
pp_coarse3d['name'] = [f'ppc{i:04d}' for i in pp_coarse3d.index.values] # ppc = pilot point coarse.

In [ ]:
pp_coarse3d[['zone','val']] = 1, 1
pp_coarse3d['z'] = pp_coarse3d['value']

In [ ]:
pp_coarse3d['layer'] = [int(i[-1]) for i in pp_coarse3d['variable']]

In [ ]:
pp_coarse3d = pp_coarse3d[['name', 'x', 'y', 'z', 'zone', 'val', 'layer']]

In [ ]:
display(pp_coarse3d)

In [ ]:
from lumpyrem import run
#MKPPSTAT
# mkppstat requires no headers in ppoint file ...sigh...
pp_coarse3d.to_csv(os.path.join(preproc_ws, 'mkppoints3d_coarse.dat'),
                       header=None, index=False, sep='\t')

In [ ]:
# Input for MKPPSTAT
# A (a)-factor of 1.5 is often reasonable. from tutorial 
npoints_h, npoints_v = 10, 10 # np-horizontal, np-vertical
a_h, a_v = 1.2, 1.2 # a-horizontal, a-vertical
# run MKPPSTAT
run.run_process(
    'mkppstat3d',
    path=preproc_ws,
    commands=['mkppoints3d_coarse.dat', npoints_h, a_h, npoints_v, a_v, 'ppstat3d_coarse.dat']
)

In [ ]:
# run PPCOV3D_SVA - pilot point covariance 3d - spatially varying anisotropy
run.run_process(
    'ppcov3d_sva',
    path=preproc_ws,
    commands=['ppstat3d_coarse.dat', 'y', 1, 'x',  'cov3d_coarse.mat', '']
)

In [ ]:
# Load cov mat file with Pyemu for further processing
import pyemu
covmat_coarse = pyemu.Cov.from_ascii(os.path.join(preproc_ws, "cov3d_coarse.mat"))

# This covaraince matrix can now be used as the base for all pilot point parameters. 
# Note that in this case the variance is 1, so it is easy to scale to a parameters prior varaince
# Depending on how you setup the scrpt, variance can be assigned at various stages (i.e. when running PPCOV_SVA, or by manipulating the matrix later)
# Note that parameter names (headers and row names) come from the parameter name sin the ppoint file. These can have a prefix added by PPCOV_SVA, or changed in the dataframe. The latter is more versatile.
covmat_coarse.to_dataframe().head()

Is this matrix only used during regularization?

In [ ]:
plt.figure(figsize=(8, 8), dpi=100)
plt.imshow(covmat_coarse.as_2d)

In [ ]:
pp_coarse3d[['kh', 'kv', 'sy', 'ss']] = 86.4, 8.64, 0.2, 0.00001

In [ ]:
pp_coarse3d

In [ ]:
pp_coarse3d.to_csv(os.path.join(preproc_ws, 'pp3d_coarse.dat'), index=False, sep='\t')

## Setup parameterization for 3D elements (K, storage, porosity)

In [ ]:
# Write generic template
with open(os.path.join(preproc_ws, 'gen_mf_array.tpl'), 'a') as f:
    f.write('$#p prop_mf.write_in_sequence(format="(1x,1pg18.11)")')

In [ ]:
def write_script(filename, lines):

    with open(filename, 'a') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

Calculate kriging factors (porosity not included yet):

In [ ]:
write_script(os.path.join(preproc_ws, 'plproc.dat'), [
f'''
### Read model grid ###
cl_mf = read_mf6_grid_specs(file=../model/{ml_name}.disv.grb,                  &
                            dimensions=2,                             &
                            slist_layer_idomain=idomain1;  layer=1,   &
                            slist_layer_idomain=idomain2;  layer=2,   &
                            slist_layer_idomain=idomain3;  layer=3)



### Read 3D pilot-points file ###
cl_pp = read_list_file(file=pp3d_coarse.dat,               &
                       id_type=character,                  &
                       dimensions=2,                       &
                       skiplines=1,                        &
                       slist=zone; col=5,                  &
                       slist=lyr; col=7,                   &
                       plist=kh_pp; col=8,                 &
                       plist=kv_pp; col=9,                 &
                       plist=sy_pp; col=10,                &
                       plist=ss_pp; col=11)


### Calculate kriging factors for each layer ###
calc_kriging_factors_auto_2d(target_clist=cl_mf, source_clist=cl_pp;select=(lyr==1), file=factors_pp_lyr1.dat)

calc_kriging_factors_auto_2d(target_clist=cl_mf, source_clist=cl_pp;select=(lyr==2), file=factors_pp_lyr2.dat)

calc_kriging_factors_auto_2d(target_clist=cl_mf, source_clist=cl_pp;select=(lyr==3), file=factors_pp_lyr3.dat)

### Write to template file ###
prop_mf=new_plist(reference_clist=cl_mf,value=1.0)

###   Horizontal K   ###
### Write kh layer 1 ###
prop_mf=86.4
prop_mf=kh_pp.krige_using_file(file='factors_pp_lyr1.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.npf_k_layer1.txt)

### Write kh layer 2 ###
prop_mf=86.4
prop_mf=kh_pp.krige_using_file(file='factors_pp_lyr2.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.npf_k_layer2.txt)

### Write kh layer 3 ###
prop_mf=86.4
prop_mf=kh_pp.krige_using_file(file='factors_pp_lyr3.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.npf_k_layer3.txt)


###    Vertical K    ###
### Write kv layer 1 ###
prop_mf=8.64
prop_mf=kv_pp.krige_using_file(file='factors_pp_lyr1.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.npf_k33_layer1.txt)

### Write kv layer 2 ###
prop_mf=8.64
prop_mf=kv_pp.krige_using_file(file='factors_pp_lyr2.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.npf_k33_layer2.txt)

### Write kv layer 3 ###
prop_mf=8.64
prop_mf=kv_pp.krige_using_file(file='factors_pp_lyr3.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.npf_k33_layer3.txt)



###      STORAGE     ###
###        sy        ###
### Write sy layer 1 ###
prop_mf=0.2
prop_mf=sy_pp.krige_using_file(file='factors_pp_lyr1.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.sto_sy_layer1.txt)


###        ss        ###
### Write ss layer 2 ###
prop_mf=0.000001
prop_mf=ss_pp.krige_using_file(file='factors_pp_lyr2.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.sto_ss_layer2.txt)

### Write ss layer 3 ###
prop_mf=0.000001
prop_mf=ss_pp.krige_using_file(file='factors_pp_lyr3.dat',transform='log')
write_model_input_file(template_file=gen_mf_array.tpl, &
    model_input_file=../model/{ml_name}.sto_ss_layer3.txt)
'''
])

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=preproc_ws,
    commands=['plproc.dat']
)

Comment out kriging factors:

In [ ]:
def replace_in_file(path, input_file, original_str, replacement_str):
    # Read in the file
    with open(os.path.join(path, input_file), 'r') as file :
        filedata = file.read()

    # Replace the target string
    filedata = filedata.replace(original_str, replacement_str)

    # Write the file out again
    with open(os.path.join(path, input_file), 'w') as file:
        file.write(filedata)

In [ ]:
# Comment out kriging factors
replace_in_file(preproc_ws, 'plproc.dat', 'calc_kriging_factors', '#calc_kriging_factors')

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=preproc_ws,
    commands=['plproc.dat']
)

### Write template file

In [ ]:
import copy
pp_coarse3d_tpl = copy.deepcopy(pp_coarse3d)

In [ ]:
pp_coarse3d_tpl

In [ ]:
pp_coarse3d_tpl['kh'] = [f'$kh{r.layer}pp{i + 1:03d}$' for i,r in pp_coarse3d_tpl.iterrows()]
pp_coarse3d_tpl['kv'] = [f'$kv{r.layer}pp{i + 1:03d}$' for i,r in pp_coarse3d_tpl.iterrows()] 
pp_coarse3d_tpl['sy'] = [f'$sy{r.layer}pp{i + 1:03d}$' for i,r in pp_coarse3d_tpl.iterrows()]
pp_coarse3d_tpl['ss'] = [f'$ss{r.layer}pp{i + 1:03d}$' for i,r in pp_coarse3d_tpl.iterrows()]

In [ ]:
pp_coarse3d_tpl

In [ ]:
pp_coarse3d_tpl.to_csv(os.path.join(template_ws, 'pp3d_coarse.tpl'), index=False, sep='\t')

In [ ]:
def line_prepender(filename, line):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)

In [ ]:
line_prepender(os.path.join(template_ws, 'pp3d_coarse.tpl'), 'ptf $')

## Setup parameterization for SFR
Setup pilot points for linear boundary conditions General Head Boundaries (GHBs) and Creek Örbäcken (SFR), starting with SFR:

In [ ]:
segfile = pd.read_csv(os.path.join(preproc_ws, 'sfr_segfile.dat'), sep='\t', names=['x', 'y', 'seg'])
display(segfile)

Fix the segments. This should probably be done in a better way and perhaps changed after the first round of history matching to get a better representation of stream reaches. This solution is just to make things work.

In [ ]:
def largest_divisor(n):
    a = 1
    for i in range(2, n):
        if n % i == 0:
            a = i
    return a

In [ ]:
nseg = largest_divisor(len(segfile))
#nseg = largest_divisor(nseg) # divide again to reduce the amount of pps from 135 to 45 (perhaps a stupid move)
print(nseg)

In [ ]:
rows_per_seg = int(len(segfile) / nseg)
print(rows_per_seg)

In [ ]:
for i in range(nseg):
    segfile.loc[(i * rows_per_seg):((i + 1) * rows_per_seg), ('seg')] = f's{i + 1}'

In [ ]:
segfile

Rewrite seglist:

In [ ]:
segfile.to_csv(
    os.path.join(preproc_ws, 'sfr_segfile.dat'),
    header=None,
    index=False,
    sep='\t',
    float_format='%.3f'
)

In [ ]:
write_script(os.path.join(preproc_ws, 'plproc_seglist_temp.dat'), [
f'''
### Read model grid ###

cl_mf = read_mf6_grid_specs(  &
    file=../model/{ml_name}.disv.grb,  &
    dimensions=3,             & # Note 3D in this case
    slist_layernum = layer,   &
    slist_idomain = idomain   &
    )


### Read Creek Örbäcken SFR seglist file ###
sl_sfr = read_segfile(file="sfr_segfile.dat", protocol=table)

### Create clist with sl_sfr as its base ###
cl_sfr_pp = create_clist_from_seglist(seglist=sl_sfr, linkage_type=endpoints, dist_thresh=5.0)

### Write reports (pps will be constructed based on theses reports) ###
cl_sfr_pp.report_dependent_lists(file='report_sfr_seglist.dat')
'''
])

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=preproc_ws,
    commands=['plproc_seglist_temp.dat']
)

In [ ]:
sfr_pps = pd.read_csv(os.path.join(preproc_ws, 'report_sfr_seglist.dat'), skiprows=[0, 1, 2])

In [ ]:
sfr_pps

In [ ]:
sfr_pps.columns = sfr_pps.columns.str.replace(r'\ (?= *?\ )', '')

In [ ]:
sfr_pps.iloc[:,0] = sfr_pps.iloc[:,0].str.replace(r'\ (?= *?\ )', '')

In [ ]:
sfr_pps[['Index', 'Int_ID', 'X-coord', 'Y-coord']] = sfr_pps['Index Int_ID X-coord Y-coord'].str.split(' ', 3, expand=True)

In [ ]:
sfr_pps['cond'] = 86.4

In [ ]:
sfr_pps.rename(columns={'Int_ID': 'ppid', 'X-coord': 'x', 'Y-coord': 'y'}, inplace=True)

In [ ]:
sfr_pps = sfr_pps[['ppid', 'x', 'y', 'cond']]

In [ ]:
display(sfr_pps)

In [ ]:
sfr_pps.to_csv(
    os.path.join(preproc_ws, 'sfrpp.dat'),
    index=False,
    sep='\t',
)

#### Parameterize the INFLOW parameter of reach 1

In [ ]:
sfr_pkgdatafile = f'{ml_name}.sfr_packagedata.txt'
shutil.copyfile(os.path.join(model_ws, sfr_pkgdatafile), os.path.join(preproc_ws, sfr_pkgdatafile))

In [ ]:
os.rename(
    os.path.join(preproc_ws, f'{ml_name}.sfr_packagedata.txt'),
    os.path.join(preproc_ws, f'{ml_name}.sfr_packagedata.tpl')
)

In [ ]:
sfr_perdatafile = f'{ml_name}.sfr_perioddata_1.txt'
shutil.copyfile(os.path.join(model_ws, sfr_perdatafile), os.path.join(template_ws, sfr_perdatafile))

In [ ]:
os.rename(
    os.path.join(template_ws, f'{ml_name}.sfr_perioddata_1.txt'),
    os.path.join(template_ws, f'{ml_name}.sfr_perioddata_1.tpl')
)

In [ ]:
line_prepender(os.path.join(template_ws, f'{ml_name}.sfr_perioddata_1.tpl'), 'ptf $')

In [ ]:
with open(os.path.join(template_ws, f'{ml_name}.sfr_perioddata_1.tpl'), 'r') as file:
    sfr_perdata = file.readlines() #pestfile partial parameters

In [ ]:
temp_row = sfr_perdata[2].split()
display(temp_row)

In [ ]:
temp_row[-1] = '$sfr_inflow1$\n'
display(temp_row)

In [ ]:
new_row = ' '.join(temp_row)
display(new_row)

In [ ]:
sfr_perdata[2] = new_row

In [ ]:
with open(os.path.join(template_ws, f'{ml_name}.sfr_perioddata_1.tpl'), 'w') as file:
    for line in sfr_perdata:
        file.write(line)

#### Write script

In [ ]:
write_script(os.path.join(preproc_ws, 'plproc_sfr.dat'), [
f'''
### Read model grid ###

cl_mf = read_mf6_grid_specs(  &
    file=../model/{ml_name}.disv.grb,  &
    dimensions=3,             & # Note 3D in this case
    slist_layernum = layer,   &
    slist_idomain = idomain   &
    )


### Read Creek Örbäcken SFR seglist file ###
sl_sfr = read_segfile(file="sfr_segfile.dat", protocol=table)

### Create clist with sl_sfr as its base ###
cl_sfr_pp = create_clist_from_seglist(seglist=sl_sfr, linkage_type=endpoints, dist_thresh=5.0)

### Instruct PLPROC to read the *_cond.dat list file to obtain conductance values at  ###
### pilot points by inserting the following function into the PLPROC script.          ###
read_list_file(reference_clist='cl_sfr_pp', file='sfrpp.dat', skiprows=1, plist='pp_sfr_cond';column=4)

### Instruct PLPROC to build an SLIST of model drain cells to which interpolation must take place ###
sfr_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.sfr_packagedata.txt, model_type=mf6_disv, &
    list_col_start=2, keytext_start='top_of_file', keytext_end='end_of_file')

### Calculate interpolation factors to these model cells through linear interpolation ###
calc_linear_interp_factors(source_clist=cl_sfr_pp, target_clist=cl_mf;select=(sfr_cells.ne.0), file="factors_sfr_cells.dat", search_radius=50)

### Write ###
sfr_cond=new_plist(reference_clist=cl_mf,value=0.0)

sfr_cond=pp_sfr_cond.interp_using_file(file=factors_sfr_cells.dat, transform=log)

replace_cells_in_lists(                                 &
    old_file={ml_name}.sfr_packagedata.tpl,             &
    new_file=../model/{ml_name}.sfr_packagedata.txt,    &
    model_type=mf6_disv,                                &
    list_col_start=2,                                   &
    keytext_start='top_of_file',                        &
    keytext_end='bottom_of_file',                       &
    plist=sfr_cond;column=9;action='replace'            &
    )

# -- This is for our own interest.
cl_mf.report_dependent_lists(file='report-sfr.dat')

'''
])

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=preproc_ws,
    commands=['plproc_sfr.dat']
)

In [ ]:
# Comment out kriging factors
replace_in_file(preproc_ws, 'plproc_sfr.dat', 'calc_linear_interp', '#calc_linear_interp')

Construct template file:

In [ ]:
sfr_pps_tpl = copy.deepcopy(sfr_pps)

In [ ]:
sfr_pps_tpl['cond'] = ['$sfrpp{:03d}$'.format(i) for i in range(1, len(sfr_pps_tpl) + 1)] 

In [ ]:
sfr_pps_tpl.to_csv(os.path.join(template_ws, 'sfrpp.tpl'), index=False, sep='\t')

In [ ]:
line_prepender(os.path.join(template_ws, 'sfrpp.tpl'), 'ptf $')

#### Setup SFR covariance matrix

In [ ]:
sfr_pps['zone'] = 1

In [ ]:
sfr_pps = sfr_pps[['ppid', 'x', 'y', 'zone', 'cond']]

In [ ]:
sfr_pps.to_csv( # Essentially a copy of sfrpp.dat but without column headers
    os.path.join(preproc_ws, 'mksfrpp.dat'),
    index=False,
    header=False,
    sep='\t',
)

In [ ]:
# run MKPPSTAT
run.run_process(
    'mkppstat',
    path=preproc_ws,
    commands=['mksfrpp.dat', '10', '1.5', 'sfrppstat.dat']
)

In [ ]:
# run PPCOV_SVA - pilot point covariance - spatially varying anisotropy
run.run_process(
    'ppcov_sva',
    path=preproc_ws,
    commands=['sfrppstat.dat', 'y', 0, 'x',  'cov_sfr.mat', '']
)

In [ ]:
covmat_sfr = pyemu.Cov.from_ascii(os.path.join(preproc_ws, "cov_sfr.mat"))
plt.figure(figsize=(8, 8), dpi=100)
plt.imshow(covmat_sfr.as_2d)

#### Visualize SFR pps

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('SFR pilot point locations', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

plt.scatter([float(i) for i in sfr_pps.x.values], [float(i) for i in sfr_pps.y.values], label='SFR PP')

plt.legend()
plt.show()

### GHB using pilot points

Copy the file and make an "template" of an `old_file` (is this really a template in a PEST context?)

In [ ]:
ghb_file = f'{ml_name}.ghb_stress_period_data_1.txt'
shutil.copyfile(os.path.join(model_ws, ghb_file), os.path.join(preproc_ws, ghb_file))

In [ ]:
os.rename(
    os.path.join(preproc_ws, f'{ml_name}.ghb_stress_period_data_1.txt'),
    os.path.join(preproc_ws, f'{ml_name}.ghb_stress_period_data_1.tpl')
)

In [ ]:
write_script(os.path.join(preproc_ws, 'plproc_ghb.dat'), [
f'''
# -- We read the MODFLOW 6 binary grid file. We read it as a 2D file.

cl_mf = read_mf6_grid_specs(file=../model/{ml_name}.disv.grb, &
                              dimensions=3,             &
                              slist_layernum = layer,   &
                              slist_idomain  = idomain)

# -- We now build an SLIST of drain cells to which we want interpolation to take place.

ghb_red_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='top_of_file', keytext_end='2387  ghb_orange')
    
ghb_orange_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='2387  ghb_red', keytext_end='2792  ghb_yellow')

ghb_yellow_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='2800  ghb_orange', keytext_end='2802  ghb_limegreen')

ghb_limegreen_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='3090  ghb_yellow', keytext_end='1  ghb_royalblue')

ghb_royalblue_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='3057  ghb_limegreen', keytext_end='1  ghb_blueviolet')

ghb_blueviolet_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='2802  ghb_royalblue', keytext_end='316  ghb_magenta')

ghb_magenta_cells = cl_mf.find_cells_in_lists(file=../model/{ml_name}.ghb_stress_period_data_1.txt, model_type=mf6_disv, &
    list_col_start=1, keytext_start='316  ghb_blueviolet', keytext_end='end_of_file')

# -- We next read the pilot points file and pilot point locations and values.

cl_pp = read_list_file(file='ghbpp.dat',                                          &
                       id_type='character',                                       &
                       dimensions=2,                                              &
                       skiprows=1,                                                &
                       plist='ghbcond_pp';column=4)

# -- Initialize the drain conductivity PLIST
ghbcond_mf=new_plist(reference_clist=cl_mf,value=0.0)


# -- We now interpolate from pilot points to the drain conductances.
#    We use inverse power of distance; but any interpolation method could be used.

ghbcond_mf(select=((ghb_red_cells.ne.0)&&(layer.eq.1)))=ghbcond_pp.ivd_interpolate_2d( &
                                        transform='log',                          &
                                        inv_power=2.0,                            &
                                        min_points=2,                             &
                                        max_points=20,                            &
                                        search_radius=1.0e20) # Perhaps too large?

ghbcond_mf(select=((ghb_orange_cells.ne.0)&&(layer.eq.1)))=ghbcond_pp.ivd_interpolate_2d( &
                                        transform='log',                          &
                                        inv_power=2.0,                            &
                                        min_points=2,                             &
                                        max_points=20,                            &
                                        search_radius=1.0e20)

ghbcond_mf(select=((ghb_yellow_cells.ne.0)&&(layer.eq.1)))=ghbcond_pp.ivd_interpolate_2d( &
                                        transform='log',                          &
                                        inv_power=2.0,                            &
                                        min_points=2,                             &
                                        max_points=20,                            &
                                        search_radius=1.0e20)

ghbcond_mf(select=((ghb_limegreen_cells.ne.0)&&(layer.eq.1)))=ghbcond_pp.ivd_interpolate_2d( &
                                        transform='log',                          &
                                        inv_power=2.0,                            &
                                        min_points=2,                             &
                                        max_points=20,                            &
                                        search_radius=1.0e20)

ghbcond_mf(select=((ghb_royalblue_cells.ne.0)&&(layer.eq.1)))=ghbcond_pp.ivd_interpolate_2d( &
                                        transform='log',                          &
                                        inv_power=2.0,                            &
                                        min_points=2,                             &
                                        max_points=20,                            &
                                        search_radius=1.0e20)

ghbcond_mf(select=((ghb_blueviolet_cells.ne.0)&&(layer.eq.1)))=ghbcond_pp.ivd_interpolate_2d( &
                                        transform='log',                          &
                                        inv_power=2.0,                            &
                                        min_points=2,                             &
                                        max_points=20,                            &
                                        search_radius=1.0e20)

ghbcond_mf(select=((ghb_magenta_cells.ne.0)&&(layer.eq.1)))=ghbcond_pp.ivd_interpolate_2d( &
                                        transform='log',                          &
                                        inv_power=2.0,                            &
                                        min_points=2,                             &
                                        max_points=20,                            &
                                        search_radius=1.0e20)

# -- A new model input file is written.

replace_cells_in_lists(old_file={ml_name}.ghb_stress_period_data_1.tpl,               &
                       new_file=../model/{ml_name}.ghb_stress_period_data_1.txt,      &
                       model_type=mf6_disv,                                       &
                       list_col_start=1,                                          &
                       keytext_start='top_of_file',                               &
                       keytext_end='bottom_of_file',                              &
                       plist=ghbcond_mf;column=4;action='replace')

# -- This is for our own interest.
#cl_mf6.report_dependent_lists(file='report-ghb.dat')
'''
])

In [ ]:
# run PLPROC
run.run_process(
    'plproc',
    path=preproc_ws,
    commands=['plproc_ghb.dat']
)

#### GHB covmat

In [ ]:
ghbpp_df = pd.read_csv(os.path.join(preproc_ws, 'ghbpp.dat'), sep='\t')

In [ ]:
ghbpp_df['zone'] = 1

In [ ]:
ghbpp_df = ghbpp_df[['ppid','x', 'y', 'zone', 'cond', 'bnames']]

In [ ]:
# Create one pp derivative of each ghb boundname
for bname in ghbpp_df.bnames.unique():
    ghbpp_df.loc[ghbpp_df['bnames'] == bname].to_csv(
        os.path.join(preproc_ws, f'mkghb{bname}pp.dat'),
        index=False,
        header=False,
        sep='\t',
    )

In [ ]:
for file in os.listdir(preproc_ws):
    if 'mkghb' in file:
        run.run_process(
            'mkppstat',
            path=preproc_ws,
            commands=[file, '10', '1.5', f'{file[2:-4]}_stat.dat']
        )

In [ ]:
for file in os.listdir(preproc_ws):
    if 'ghb' in file and '_stat.dat' in file:
        filename = f'cov_{file[:6]}.mat'
        run.run_process(
            'ppcov_sva',
            path=preproc_ws,
            commands=[file, 'y', 1, 'x',  filename, '']
        )

#### Visualize GHB pps

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)
ax = fig.add_subplot(1, 1, 1, aspect='equal')
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('GHB pilot point locations', fontsize=18)

mapview = flopy.plot.PlotMapView(gwf, layer=0)
linecollection = mapview.plot_grid(lw=0.25)

for bname in ghbpp_df['bnames'].unique():
    df = ghbpp_df.loc[ghbpp_df['bnames'] == bname]
    x = [float(i) for i in df.x.values]
    y = [float(i) for i in df.y.values]
    
    plt.scatter(x, y, color=bname, label=f'ghb_{bname}')

plt.legend()
plt.show()

## Parameterize LUMPREM models

Parameters to be adjusted through history-matching are:


| parameter | default | initial | upper bound | lower bound |
| --------- |--------:|--------:|-------------| ------------|
| maxvol    | 0.5     | 0.2     | 1           | 0.01        |
| rdelay    | 5       | 5       | 10          | 1           |
| mdelay    | 1       | 1       | 5           | 0           |
| Ks        | 0.1     | 86.4    | 864.0       | 0.000864    |
| m         | 0.5     | 1.0     | 2.0         | 0.01        |
| l         | 0.5     | 0.5     | 1.0         | 0.0         |
| mflowmax  | 0.1     | 0.1     | 0.2         | 0.01        |
| f         | 0.5     | 0.5     | 0.9         | 0.01        |
| γ         | 1.0     | 1.0     | 10.0        | 0.01        |

- *maxvol* The volume of the soil moisture store.
- *rdelay* Delay, in days, between water draining from the soil moisture store.
- *mdelay* Delay, in days, between water leaving the soil moisture store.
- *Ks* Hydraulic conductivity (meters per day). Initial: 86.4 (0.001 m/s)
- *m* A parameter determining the shape of the drainage rate vs. stored water relationship.
- *l* A pore-connectivity parameter (estimated by Mualem, 1976, to be about 0.5 for many soils).
- *mflowmax* Maximum macropore recharge allowed per day.
- *f* Crop factor.
- *γ* A parameter determining the shape of the evaporation rate vs. stored water relationship.

In [ ]:
def parameterize_lr_model(lr_template_file, lr_ml_name):
    # Read in the file
    with open(os.path.join(template_ws, lr_template_file), 'r') as file:
        lines = file.readlines()

    # Replace the target string
    lines[2] = f'$maxvol_{lr_ml_name}$  0 \n' #earth properties maxvolume & irrigvolfrac
    lines[3] = f'$rdelay_{lr_ml_name}$  $mdelay_{lr_ml_name}$ \n' #earth properties delay factors
    lines[4] = f'$ks_{lr_ml_name}$  $m_{lr_ml_name}$  $l_{lr_ml_name}$  $mfmax_{lr_ml_name}$  \n' #earth properties factors
    lines[24] = f'$crfac_{lr_ml_name}$  $gamma_{lr_ml_name}$  \n' #crop factor & gamma

    # Write the file out again
    with open(os.path.join(template_ws, lr_template_file), 'w') as file:
        for line in lines:
            file.write(line)

Parameterize recharge model:

In [ ]:
parameterize_lr_model('lr_lu1.tpl', 'lu1')

Parameterize GHB models:

In [ ]:
for file in os.listdir(template_ws):
    if 'lr_' in file and 'lu1' not in file:
        parameterize_lr_model(file, file[3:][:3])

Done.

## Build partial PEST control file using TPL2PST

Write tpl2pst script:

In [ ]:
write_script(os.path.join(template_ws, 'tpl2pst.in'), [
f'''
# An example TPL2PST input file

start template_file
  template_file     = pp3d_coarse.tpl
  model_input_file  = pp3d_coarse.dat
  transform         = log
  value             = 1.0
  lower_bound       = 0.1     
  upper_bound       = 10
  pargroup          = changethis
end template_file

start template_file
  template_file     = sfrpp.tpl
  model_input_file  = sfrpp.dat
  transform         = log
  value             = 86.4
  lower_bound       = 0.000864     
  upper_bound       = 864.0
  pargroup          = sfrc
end template_file

start template_file
  template_file     = ghbpp.tpl
  model_input_file  = ghbpp.dat
  transform         = log
  value             = 86.4
  lower_bound       = 0.000864     
  upper_bound       = 864.0
  pargroup          = ghbc
end template_file

start template_file
  template_file     = hagfors_1.sfr_perioddata_1.tpl
  model_input_file  = hagfors_1.sfr_perioddata_1.txt
  transform         = log
  value             = 15000
  lower_bound       = 100     
  upper_bound       = 40000
  pargroup          = sfrinfl1
end template_file

start template_file
  template_file     = lr_lu1.tpl 
  model_input_file  = lr_lu1.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = rch
end template_file

start template_file
  template_file     = lr_red.tpl 
  model_input_file  = lr_red.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = lrghbred
end template_file

start template_file
  template_file     = lr_orange.tpl 
  model_input_file  = lr_orange.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = lrghbora
end template_file

start template_file
  template_file     = lr_yellow.tpl 
  model_input_file  = lr_yellow.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = lrghbyel
end template_file

start template_file
  template_file     = lr_limegreen.tpl 
  model_input_file  = lr_limegreen.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = lrghblim
end template_file

start template_file
  template_file     = lr_royalblue.tpl 
  model_input_file  = lr_royalblue.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = lrghbroy
end template_file

start template_file
  template_file     = lr_blueviolet.tpl 
  model_input_file  = lr_blueviolet.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = lrghbblu
end template_file

start template_file
  template_file     = lr_magenta.tpl 
  model_input_file  = lr_magenta.in
  transform         = log
  value             = 1 # change this
  lower_bound       = 0.1 # change this     
  upper_bound       = 10 # change this
  pargroup          = lrghbmag
end template_file
'''
])

In [ ]:
# run TPL2PST
run.run_process(
    'tpl2pst',
    path=template_ws,
    commands=['tpl2pst.in', 'param.pst']
)

running tpl2pst doesn't work for some reason. has to be run subprocess below:

In [ ]:
import subprocess
p = subprocess.run(['tpl2pst', 'tpl2pst.in', 'param.pst'], cwd=template_ws, stdout=subprocess.PIPE, text=True)
for row in p.stdout.split('\n'):
    print(row)

In [ ]:
os.listdir(template_ws) # param.pst is now created

### Make necessary changes to param.pst:

In [ ]:
with open(os.path.join(template_ws, 'param.pst'), 'r') as file:
    pstfilep_param = file.readlines() #pestfile partial parameters

In [ ]:
pstfilep_param

In [ ]:
print('Relevant indices:') # print relevant indices of the pstfilep_param
print(pstfilep_param.index('* parameter groups\n'))
print(pstfilep_param.index('* parameter data\n'))
print(pstfilep_param.index('* observation groups\n'))

### Change parameter data

Let's start by changing all the parameters marked with "changethis" (as suggested by calibration tutorial):

In [ ]:
paramdata_start = pstfilep_param.index('* parameter data\n') + 1
paramdata_stop = pstfilep_param.index('* observation groups\n')

In [ ]:
pardata_df = pd.DataFrame(pstfilep_param[paramdata_start:paramdata_stop]) # read df

In [ ]:
pardata_df.iloc[:,0] = pardata_df.iloc[:,0].str.replace(r'\ (?= *?\ )', '') # clean up data structure

In [ ]:
pardata_cols = ['PARNME', 'PARTRANS', 'PARCHGLIM', 'PARVAL1', 'PARLBND', 'PARUBND', 'PARGP', 'SCALE', 'OFFSET']

In [ ]:
pardata_df[pardata_cols] = pardata_df[0].str.split(' ', 9, expand=True) # add columns

In [ ]:
pardata_df = pardata_df[pardata_cols] # remove redundant column
display(pardata_df) # view df

Sort the dataframe based on parameters (as suggested by calibration tutorial):

In [ ]:
pardata_df = pardata_df.sort_values(['PARNME', 'PARGP']).reset_index(drop=True)

Assign new parameter groups (PARGP) for each type of parameter:

In [ ]:
parnme_pargrp = [
    'ghbcblu', 'ghbclim', 'ghbcmag', 'ghbcora', 'ghbcred', 
    'ghbcroy','ghbcyel', 'kh1pp', 'kh2pp', 'kh3pp', 'kv1pp',
    'kv2pp', 'kv3pp', 'sfrpp', 'ss1pp', 'ss2pp', 'ss3pp',
    'sy1pp', 'sy2pp', 'sy3pp'
]

In [ ]:
for i in parnme_pargrp:
    pardata_df.loc[pardata_df['PARNME'].str.contains(i), 'PARGP'] = i

In [ ]:
pardata_df

Assign new (initial) parameter value, lower and upper bounds for the non-linear pilot point parameters:

In [ ]:
parval1 = {'kh': 86.4, 'kv': 8.64, 'sy': 0.2, 'ss': 0.000001}
parlbnd = {'kh': 8.64E-04, 'kv': 8.64E-05, 'sy': 0.05, 'ss': 0.00000001}
parubnd = {'kh': 864.0, 'kv': 864.0, 'sy': 0.35, 'ss': 0.001}

In [ ]:
for d, col in zip([parval1, parlbnd, parubnd], ['PARVAL1', 'PARLBND', 'PARUBND']):
    for k,v in d.items():
        pardata_df.loc[pardata_df['PARGP'].str.contains(k), col] = v

In [ ]:
display(pardata_df)

Assign new (initial) parameter value, lower and upper bounds for the **lumprem** parameters:

Extract data from df to list:

In [ ]:
pardata_list = pardata_df.values.tolist()

In [ ]:
pardata_new = []
for i in pardata_list:
    pardata_new.append(' '.join(str(v) for v in i))

This is the old parameter data:

In [ ]:
display(pstfilep_param[paramdata_start:paramdata_stop])

Which will now be replaced by the new parameter data, which is this:

In [ ]:
display(pardata_new) # show list

Create a temporary copy of the original `pstfilep_param`:

In [ ]:
pstfilep_param_temp = pstfilep_param[:]

Replace the old parameter data with the changed pardata in the templist:

In [ ]:
for (index, replacement) in zip([i for i in range(paramdata_start, paramdata_stop)], pardata_new):
    pstfilep_param_temp[index] = replacement

The parameter data has now been replaced:

In [ ]:
display(pstfilep_param_temp)

### Change parameter groups

Get relevant indices:

In [ ]:
pargrp_start = pstfilep_param_temp.index('* parameter groups\n') + 1
pargrp_stop = pstfilep_param_temp.index('* parameter data\n')

Let's have a look at the current parameter grous config:

In [ ]:
pargrp_df = pd.DataFrame(pstfilep_param_temp[pargrp_start:pargrp_stop])
pargrp_df.iloc[:,0] = pargrp_df.iloc[:,0].str.replace(r'\ (?= *?\ )', '')
pargrp_cols = ['PARGPNME', 'INCTYP', 'DERINC', 'DERINCLB', 'FORCEN', 'DERINCMUL', 'DERMTHD']
pargrp_df[pargrp_cols] = pargrp_df[0].str.split(' ', 7, expand=True)
pargrp_df = pargrp_df[pargrp_cols]
display(pargrp_df)

Let's recreate this data structure (no need to use a dataframe):

In [ ]:
pargrp_new = [' '.join(str(v) for v in [i, 'relative', 0.015, 0.0, 'switch', 2, 'parabolic\n']) for i in pardata_df['PARGP'].unique()]
display(pargrp_new)

Make a new copy of the partial pestfile:

In [ ]:
pstfilep_param_new = pstfilep_param_temp[:]

Replace the old parameter groups with the changed pargp in the temp2list:

In [ ]:
pstfilep_param_new[pargrp_start:pargrp_stop] = pargrp_new

The parameter groups have now been changed:

In [ ]:
display(pstfilep_param_new)

### Change NPARGP (number of parameter groups)

This is the third column of the third index (number 3 below):

In [ ]:
pstfilep_param_new[3]

In [ ]:
temp_row = pstfilep_param_new[3].split()
display(temp_row)

In [ ]:
npargp = len(pardata_df['PARGP'].unique()) # number of parameter groups
print(npargp)

In [ ]:
temp_row[2] = str(npargp)
temp_row[-1] = '0\n'
display(temp_row)

In [ ]:
new_row = ' '.join(temp_row)
display(new_row)

In [ ]:
pstfilep_param_new[3] = new_row

In [ ]:
display(pstfilep_param_new)

### Write new partial pest parameter file:

In [ ]:
with open(os.path.join(template_ws, 'partial_param.pst'), 'w') as file:
    for line in pstfilep_param_new:
        file.write(line)

### Copy pestfile to pestdir

In [ ]:
partial_pestfile = 'partial_param.pst'
shutil.copyfile(os.path.join(template_ws, partial_pestfile), os.path.join(pest_ws, partial_pestfile))

In [ ]:
os.rename(
    os.path.join(pest_ws, 'partial_param.pst'),
    os.path.join(pest_ws, 'calib0.pst')
)

### Fix model input/output

In [ ]:
with open(os.path.join(pest_ws, 'calib0.pst'), 'r') as file:
    pstfilep_calib = file.readlines() #pestfile partial parameters

Get index:

In [ ]:
ml_io_start = pstfilep_calib.index('* model input/output\n') + 1

In [ ]:
pstfilep_calib[ml_io_start:]

In [ ]:
prepend1, prepend2 = '.\\template\\', '..\\runmodel\\preproc\\'

In [ ]:
new_ml_io = []
for i in pstfilep_calib[ml_io_start:]:
    for string, j in zip([prepend1, prepend2], i.split()):
        new_ml_io.append(string + j)

In [ ]:
new_ml_io

In [ ]:
new_ml_io = [' '.join(i) for i in zip(new_ml_io[0::2], new_ml_io[1::2])]

In [ ]:
new_ml_io = [i+'\n' for i in new_ml_io]

In [ ]:
new_ml_io

In [ ]:
pstfilep_calib[ml_io_start:] = new_ml_io

Overwrite `calib0.pst`:

In [ ]:
with open(os.path.join(pest_ws, 'calib0.pst'), 'w') as file:
    for line in pstfilep_calib:
        file.write(line)

In [ ]:
hds_meas_lowpass

Parameters: kh, kx, sy, ss, sfrcond, ghbcond, lu1 lumprem recharge model (still tbd ghb lumprem models)

### END OF NOTEBOOK

---